In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
!ls

data  subs  titanic.ipynb


In [3]:
!ls data

test.csv  train.csv


In [4]:
train = pd.read_csv('data/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train['Age'].mean(skipna=True)

29.69911764705882

In [6]:
def preprocessing(data):
    data.fillna({'Embarked':'_', 'Age':data['Age'].mean(skipna=True)}, inplace=True)
    enc = OneHotEncoder()
    enc.fit(data[['Sex', 'Embarked', 'Pclass']].values)
    one_hot = enc.transform(data[['Sex', 'Embarked', 'Pclass']].values).toarray().astype(int)
    data['Female'] = one_hot[:,0] 
    data['Male'] = one_hot[:,1]
    data['Cherbourg_port'] = one_hot[:,2]
    data['Queenstown_port'] = one_hot[:,3]
    data['Southampton_port'] = one_hot[:,4]
    data['1_class'] = one_hot[:,6]
    data['2_class'] = one_hot[:,7]
    data['3_class'] = one_hot[:,8]
    data['Age'] = MinMaxScaler().fit_transform(data['Age'].values.reshape(-1, 1))
    data['SibSp'] = MinMaxScaler().fit_transform(data['SibSp'].values.reshape(-1, 1))
    data['Parch'] = MinMaxScaler().fit_transform(data['Parch'].values.reshape(-1, 1))
    data['Fare'] = MinMaxScaler().fit_transform(data['Fare'].values.reshape(-1, 1))
    

In [7]:
preprocessing(train)

In [8]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Female,Male,Cherbourg_port,Queenstown_port,Southampton_port,1_class,2_class,3_class
0,1,0,3,"Braund, Mr. Owen Harris",male,0.271174,0.125,0.0,A/5 21171,0.014151,NaN,S,0,1,0,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.472229,0.125,0.0,PC 17599,0.139136,C85,C,1,0,1,0,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,0.321438,0.000,0.0,STON/O2. 3101282,0.015469,NaN,S,1,0,0,0,1,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.434531,0.125,0.0,113803,0.103644,C123,S,1,0,0,0,1,1,0,0
4,5,0,3,"Allen, Mr. William Henry",male,0.434531,0.000,0.0,373450,0.015713,NaN,S,0,1,0,0,1,0,0,1


In [9]:
clf = LogisticRegression()

In [10]:
X = train[[
        'Age', 'SibSp', 'Parch', 'Fare', 'Female', 'Male',
        'Cherbourg_port', 'Queenstown_port', 'Southampton_port',
        '1_class', '2_class', '3_class'
    ]].values
y = train['Survived']

In [11]:
scores = cross_val_score(
    clf, 
    X,
    y,
    cv=5
)


/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/si

In [12]:
print(scores)

[0.7877095  0.81005587 0.78651685 0.75842697 0.81355932]


In [14]:
clf.fit(X, y)

/home/svasilyev/anaconda3/envs/technosphere/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
clf.score(X, y)

0.8013468013468014

In [16]:
test = pd.read_csv('data/test.csv')

In [17]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
